In [153]:
import numpy as np
import copy
from time import time
# from numba import njit, prange
import matplotlib.pyplot as plt
from matplotlib import text
%matplotlib inline
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, InsetPosition, mark_inset
from scipy import special
import pandas as pd

## From Cartesian coordinate to spherical coordiante
This part defines the transformation between the Cartesian coordinate $(x, y, z)$ and the spherical coordiante $(r, \theta, \phi)$:
\begin{aligned}
x&=r\sin\theta\cos\phi \\
y&=r\sin\theta\sin\phi \\
z&=r\cos\theta
\end{aligned}
and the spherical coordinate can be obtained from
\begin{aligned}
r&=\sqrt{x^2+y^2+z^2} \\
\theta&=\arccos\left(\frac{z}{r}\right) \\
\phi&=\arctan\left(\frac{y}{x}\right)
\end{aligned}
with a modification of the definition of the $\arctan$ function in $\phi$ to guarantee that
\begin{aligned}
r&\in[0, +\infty) \\
\theta&\in[0, \pi] \\
\phi& \in[0, 2\pi]
\end{aligned}

Note that the definitions of $\theta$ and $\phi$ are opposite to those in the spherical harmonics function in scipy.

In [100]:
def xyz2sph(x):
    r = np.sqrt((x**2).sum())
    theta = np.arccos(x[2]/r)
    phi = (np.arctan2(x[1], x[0]))%twopi
    return np.array([r, theta, phi])

In [103]:
def xyz2sph_array(x):
    r = np.sqrt((x**2).sum(axis=1))
    theta = np.arccos(x[:, 2]/r)
    phi = (np.arctan2(x[:, 1], x[:, 0]))%twopi
    return np.c_[r, theta, phi]

## Read the initial configuration

In [105]:
Natom = 0
lo = []
hi = []
x0 = []
with open("init.xyz", "r") as f:
    for i in range(3):
        f.readline()
    Natom = int(f.readline().split()[0])
    f.readline()
    for i in range(3):
        line = f.readline().split()
        lo.append(float(line[0]))
        hi.append(float(line[1]))
    f.readline()
    for i in range(Natom):
        x0.append(f.readline().split()[2:])
lo = np.array(lo)
hi = np.array(hi)
prd = hi - lo
x0 = np.array(x0, dtype = float)

In [327]:
Natom = 0
lo = []
hi = []
x_fcc = []
with open("fcc.xyz", "r") as f:
    for i in range(3):
        f.readline()
    Natom = int(f.readline().split()[0])
    f.readline()
    for i in range(3):
        line = f.readline().split()
        lo.append(float(line[0]))
        hi.append(float(line[1]))
    f.readline()
    for i in range(Natom):
        x_fcc.append(f.readline().split()[2:])
lo = np.array(lo)
hi = np.array(hi)
prd_fcc = hi - lo
x_fcc = np.array(x_fcc, dtype = float)

In [329]:
x_fcc.shape

(256, 3)

In [330]:
prd_fcc

array([5.54689019, 5.54689019, 5.54689019])

In [331]:
2.7734450974025386*2

5.546890194805077

## Distance under the periodic boundary condition

In [19]:
def pbc_dist(dist, prd):
    s = dist / prd
    return (s - np.floor(s+0.5))*prd

In [128]:
nharm = 6
marray = np.arange(-nharm, nharm+1)

In [129]:
marray

array([-6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6])

In [333]:
2.77345-2.08008

0.6933699999999998

In [315]:
def q6i(x, i, prd):
    xi = x[i]
    q6i_vec = np.zeros(13)
    xjs = np.delete(x, i, axis=0)
    dxs = pbc_dist(xjs-xi, prd)
    rijs = np.sqrt((dxs**2).sum(axis=1))
    dxs_cut = dxs[rijs <= 1.5]
    rijs = np.insert(rijs, i, 0)
    neighi = np.array(np.where(rijs<=1.5))
    neighi = neighi[neighi!=i]
#     print(dxs_cut)
    sph = xyz2sph_array(dxs_cut)
#     print(sph)
#     print(sph[:, 0])
#     print(sph[:, 1:]/np.pi)
    theta = sph[:, 1]
    phi = sph[:, 2]
    Ynmj = np.zeros([2*nharm+1, dxs_cut.shape[0]], dtype="complex")
    for mn in range(2*nharm+1):
        Ynmj[mn] = special.sph_harm(marray[mn], nharm, phi, theta)
#     print(Ynmj)
#     print(Ynmj.shape)
#     Ynm = special.sph_harm(1, nharm, phi, theta)
    if Ynmj.shape[1]!=0:
        Ynm = Ynmj.mean(axis=1)
    else:
        Ynm = np.zeros(2*nharm+1)
#     print(Ynmj[:, -1])
    return Ynm, neighi

In [317]:
def q6(x, prd):
    natom = x.shape[0]
    q6is = np.zeros([natom, 2*nharm+1], dtype="complex")
    neighbors = []
    for i in range(natom):
        q6is[i], neighi = q6i(x, i, prd)
#         print(neighi)
        neighbors.append(np.array(neighi))
    return q6is, neighbors

In [368]:
def nbond(x, prd):
    natom = x.shape[0]
    nb = np.zeros(natom)
    q6is, neighbors = q6(x, prd)
    for i in range(natom):
        qdot = np.dot(q6is[neighbors[i]], np.conjugate(q6is[i]))
        qdotr = qdot.real
        nb[i] = (qdotr[qdotr>=0.65]).shape[0]
    return nb

In [285]:
df = pd.read_excel('r_12.xlsx', header=None)
xqj = np.array(df)
prd_qj = np.array([4, 12, 30])

In [258]:
df = pd.read_excel('r_27.xlsx', header=None)
x27 = np.array(df)
prd_27 = np.array([4, 12, 30])

In [378]:
rs = 1.5

In [385]:
# def cluster_detect(x, prd):
#     x0 = x[0]
# #     xjs = np.delete(x, i, axis=0)
#     dxs = pbc_dist(x-x0, prd)
#     rijs = np.sqrt((dxs**2).sum(axis=1))
#     neigh0 = np.where(rijs <= rs)[0]
#     clustered = np.array([neigh0], dtype="object")
#     for ii in clustered[0][1:]:
#         n 
#     return clustered

In [386]:
cluster_detect(xqj, prd_qj)

array([[0, 5, 44, 60, 77, 101, 121, 168, 190, 210, 211, 248, 266, 281,
        308, 324, 326]], dtype=object)

In [645]:
def DFS(i, k, x, prd, label):
#     print("i=%d"%(i))
    natom = x.shape[0]
    label[i] = k
    
    xi = x[i]
    dxs = pbc_dist(x-xi, prd)
    rijs = np.sqrt((dxs**2).sum(axis=1))
    
    idx = np.arange(natom)
    idx0 = idx[idx!=i]
#     print(idx0)
#     unvisited = idx0[label[idx0]==0]
    for j in idx0:
        if label[j] != 0:
            continue
        else:
            dx = pbc_dist(x[j]-x[i], prd)
#             rij = np.sqrt((dx**2).sum())
            rij = rijs[j]
            if rij <= 1.5:
#                 if i==4 or i==15:
#                 print(i, j, k, rij)
                label = DFS(j, k, x, prd, label)
#             neigh_uv = unvisited[rijs[unvisited]<=1.5]
#     print(neigh_uv)
#     print(len(neigh_uv))
#     if(len(neigh_uv)>0):
#         for j in neigh_uv:
#             print("len = %d, j=%d"%(len(neigh_uv), j))
#             label = DFS(j, k, x, prd, label)
#     print("after if, len=%d"%(len(neigh_uv)))
    return label

In [617]:
label0 = np.zeros(xqj.shape[0])

In [618]:
label_qj = DFS(0, 2, xqj, prd_qj, label0)

In [619]:
np.where(label_qj==2)[0]+1

array([  1,   6,  45,  47,  61,  78, 102, 122, 169, 191, 211, 212, 249,
       267, 282, 295, 309, 312, 325, 327])

In [646]:
def cluster_detect(x, prd):
    natom = x.shape[0]
    label = np.zeros(natom)
    k = 0
#     print(label)
    for i in range(natom):
        if label[i] == 0:
            k += 1
#             print(i)
            label = DFS(i, k, x, prd, label)
    return label

In [655]:
def cluster_size(x, prd):
    label = cluster_detect(x, prd)
    ncluster = int(np.max(label))
    sizes = np.zeros(ncluster)
    for i in range(ncluster):
        sizes[i] = np.count_nonzero(label[label==i+1])
    return sizes

In [641]:
np.sqrt(((xqj[253]-xqj[15])**2).sum())

10.988241396512084

In [633]:
np.sqrt(((pbc_dist(xqj[253]-xqj[15], prd_qj))**2).sum())

1.365965897923961

In [636]:
xqj[253]

array([-1.50717081, -5.09099456, 11.00868107])

In [637]:
xqj[15]

array([-1.28776139,  5.86215486, 10.15907751])

In [647]:
label_qj = cluster_detect(xqj, prd_qj)

In [656]:
np.sort(cluster_size(xqj, prd_qj))

array([ 1.,  1.,  1.,  1.,  1.,  1.,  2.,  4.,  4.,  8.,  9., 10., 10.,
       11., 12., 12., 13., 14., 16., 20., 20., 20., 21., 28., 31., 33.,
       39.])

In [657]:
np.max(cluster_size(xqj, prd_qj))

39.0

In [658]:
len(cluster_size(xqj, prd_qj))

27

In [659]:
np.max(cluster_size(x27, prd_27))

7.0

In [661]:
len(ç)

8

In [660]:
cluster_detect(x27, prd_27)

array([1., 2., 3., 4., 4., 3., 2., 2., 5., 6., 7., 7., 8., 8., 6., 7., 7.,
       7., 2., 1., 4., 2., 2., 2., 3., 3., 3.])

In [662]:
cluster_size(x27, prd_27)

array([2., 7., 5., 3., 1., 2., 5., 2.])

In [563]:
label_qj[15]

4.0

In [544]:
idx4

array([  4,  15,  25,  42,  63,  66,  75,  79,  96, 104, 112, 130, 131,
       139, 149, 214, 223, 224, 228, 249, 250, 253, 261, 267, 273, 277,
       291, 292, 315, 322, 334])

In [545]:
for i in neigh15:
    print(i, np.sqrt((pbc_dist(xqj[15]-xqj[i], prd_qj)**2).sum()))

66 1.329674225531355
139 0.9061592195213352
223 0.7680769508537045
228 1.0506262308996785
253 1.365965897923961
261 1.447769706220097
273 0.6497693625297983
277 0.9683138578049645
322 0.8272187115167473


In [546]:
idx4 = np.where(label_qj==4)[0]

In [430]:
prd_qj

array([ 4, 12, 30])

In [444]:
idx4_ = idx4[idx4!=15]
xj4 = xqj[idx4_]
dxs = pbc_dist(xj4-xqj[15], prd_qj)
rijs = np.sqrt((dxs**2).sum(axis=1))
neigh15 = idx4_[np.where(rijs<1.5)]

In [451]:
idx4_ = idx4[idx4!=4]
xj4 = xqj[idx4_]
dxs = pbc_dist(xj4-xqj[4], prd_qj)
rijs = np.sqrt((dxs**2).sum(axis=1))
neigh4 = idx4_[np.where(rijs<1.5)]

In [455]:
idx4_ = idx4[idx4!=25]
xj4 = xqj[idx4_]
dxs = pbc_dist(xj4-xqj[25], prd_qj)
rijs = np.sqrt((dxs**2).sum(axis=1))
neigh25 = idx4_[np.where(rijs<1.5)]

In [456]:
neigh25

array([  4,  42,  63,  75, 104, 112, 130, 131, 149, 224, 249, 250, 267,
       291, 292, 315, 334])

In [452]:
neigh4

array([ 25,  42,  63,  75,  96, 104, 112, 149, 224, 250, 292, 315, 334])

In [454]:
neigh15

array([ 66, 139, 223, 228, 253, 261, 273, 277, 322])

In [435]:
rijs.shape

(30,)

In [438]:
rijs

array([3.26743417, 3.41213978, 2.7446168 , 2.9579712 , 1.32967423,
       3.27957405, 2.1869734 , 2.23359601, 2.16675739, 3.18743038,
       2.70501053, 4.43143243, 0.90615922, 3.33208046, 1.67330398,
       0.76807695, 3.49434485, 1.05062623, 2.49493107, 3.2808227 ,
       1.3659659 , 1.44776971, 3.94134956, 0.64976936, 0.96831386,
       2.49493616, 3.59239738, 3.54699493, 0.82721871, 2.85644189])

In [439]:
(rijs[rijs<1.5]).shape

(9,)

array([ 66, 139, 223, 228, 253, 261, 273, 277, 322])

In [420]:
rijs[rijs<1.5]

array([1.32967423, 0.90615922, 0.6740346 , 1.05062623, 1.21109676,
       1.44776971, 0.64976936, 0.96831386, 0.82721871])

In [417]:
idx4[idx4!=15]

array([  4,  25,  42,  63,  66,  75,  79,  96, 104, 112, 130, 131, 139,
       149, 214, 223, 224, 228, 249, 250, 253, 261, 267, 273, 277, 291,
       292, 315, 322, 334])

In [414]:
idx4[0]

array([  4,  15,  25,  42,  63,  66,  75,  79,  96, 104, 112, 130, 131,
       139, 149, 214, 223, 224, 228, 249, 250, 253, 261, 267, 273, 277,
       291, 292, 315, 322, 334])

39.0